In [3]:
'''
GEOG5790 - Programming for Geographical Information Analysis: Advanced Skills
Independent Project - EA WIMS Water Quality Data Analyser/Viewer

Anne Harding (200754573)
05/05/2019

DataViewer.ipynb

Jupyter notebook for user to view and analyse data for user-specified .csv file
containing WQ data (using Text widget for user input) and for a specified determinand
(using Dropdown widget for user input).

This code is split into 3 steps:
Step 1)  Import modules and load input file for analysis.
Step 2)  Confirm input file is appropriate for analysis and display Dropdown widget
         for user to select determinand for analysis.
Step 3)  Prepare data for plotting, calculate statistics, produce plot and map.

The user can run this code in 3 steps by clicking inside each code cell and pressing
SHIFT + ENTER. The following will happen when each cell is run:
Step 1)  A text box will appear in which the user should enter the filepath of the 
         water quality data file they would like to analyse.
Step 2)  A dropdown menu containing the list of determinands included in the input
         datafile. The user should select the determinand which they wish to perform
         analysis for.
Step 3)  The user will obtain 3 outputs:
            1 - A map plotting each of the sampling points used in the analysis.
            2 - An interactive plot graphing the sampling data. The user may filter
                the date range using the plot, and choose to select/deselect traces.
            3 - A table containing descriptive statistics for the sampling data.
         Note that the map, plot and table will be saved within a "plots" 
         subdirectory in the directory containing the input .csv datafile.
'''
# -----------------------------------------------------------------------------------
# STEP ONE: IMPORT MODULES AND LOAD INPUT FILE FOR ANALYSIS.
# -----------------------------------------------------------------------------------

%matplotlib inline

# Import modules:
import os
import pandas as pd
import numpy as np
import folium
from matplotlib import pyplot
import ipywidgets as widgets
from IPython.display import display
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
from convertbng.util import convert_lonlat

# Set up Text widget for user to add filepath for datafile:
file_input = widgets.Text(
    value='...',
    description='Data file:',
    disabled=False
)
# Display widget:
print("Please select .csv file for analysis:")
display(file_input)

# Note: Would like a file browser window here, but I don't think a widget exists to do so in Jupyter.

Please select .csv file for analysis:


Text(value='...', description='Data file:')

In [20]:
# -----------------------------------------------------------------------------------
# STEP TWO: CONFIRM INPUT FILE IS APPROPRIATE FOR ANALYSIS AND DISPLAY DROPDOWN
# WIDGET FOR USER TO SELECT DETERMINAND FOR ANALYSIS.
# -----------------------------------------------------------------------------------

# Get path for input data file from Text widget:
datafile = file_input.value

# Check if file exists:
if os.path.isfile(datafile):
    # Get file extension:
    filename, extension = os.path.splitext(datafile)
    # Print statement to manually check file extension:
    # print(extension)
    # Check if file extension is .csv:
    if extension != '.csv':
        raise ValueError("File must be .csv format.")
    else:
        print("Input data file accepted.")
        # Read .csv file into a pandas dataframe:
        print("Reading data.")
        df = pd.read_csv(datafile)
        # Print statement to manually check dataframe:
        # print(df)
        # Get list of unique determinands from dataframe:
        dets = df["determinand.definition"].unique()
        # Print statement to manually check unique list of determinands:
        # print(dets)
else:
    raise ValueError("File does not exist.")

# Set up Dropdown widget for user to choose determinand of interest:
det_dd = widgets.Dropdown(
    options=dets,
    value=dets[0],
    description='Determinand:',
    disabled=False,
)
# Display widget:
print("Please select determinand of interest:")
display(det_dd)

Input data file accepted.
Reading data.
Please select determinand of interest:


Dropdown(description='Determinand:', options=('Zinc', 'Orthophosphate, reactive as P', 'Temperature of Water',…

In [25]:
# -----------------------------------------------------------------------------------
# STEP THREE: PREPARE DATA FOR PLOTTING, CALCULATE STATISTICS, PRODUCE PLOT AND MAP.
# -----------------------------------------------------------------------------------
# PREPARE DATA:

# Obtain chosen determinand for analysis from Dropdown widget:
chosen_det = det_dd.value
# Print statement to manally check chosen determinand:
# print(chosen_det)

# Filter dataframe to only keep values for chosen determinand:
df_filtered = df[df['determinand.definition'] == chosen_det]
# Print statement to check filtered dataframe:
# print(df_filtered)

# Get standard units for chosen determinand:
units = df_filtered["determinand.unit.label"].unique()[0]
# Print statement to manually check units:
# print(units)

# Get list of sampling points from filtered dataframe:
locs = df_filtered["sample.samplingPoint.notation"].unique()
# Print statement to manually check unique list of sampling points:
# print(locs)

# Sort df_filtered by date:
df_ordered = df_filtered.sort_values(by='sample.sampleDateTime')
# print(df_ordered)

# -----------------------------------------------------------------------------------
# DIRECTORY AND FILENAMES FOR SAVING PLOTS:

# Get directory of input file:
dir = os.path.dirname(datafile)
# Define path for subdirectory to save plots:
plots_dir = os.path.join(dir, "plots")

# Try to create output directory:
try:
    os.mkdir(plots_dir)
    print("Directory {} created.".format(plots_dir))
# If output directory already exists, raise FileExistsError:
except FileExistsError:
    # print("Directory {} already exists.".format(plots_dir))
    pass

# Create filename for saving plot:
plot_filename = os.path.join(plots_dir, chosen_det + "_plot.html")
# Create filename for saving map:
map_filename = os.path.join(plots_dir, chosen_det + "_map.html")
# Create filename for saving table:
tbl_filename = os.path.join(plots_dir, chosen_det + "_stats.csv")

# -----------------------------------------------------------------------------------
# DATA PLOTTING:

# Plot data and save as .html file using filename:
fig = {
    'data': [
        {
            'x': df_ordered[df_ordered['sample.samplingPoint.notation']
                             ==loc]['sample.sampleDateTime'],
            'y': df_ordered[df_ordered['sample.samplingPoint.notation']
                             ==loc]['resultQualified'],
            'name': loc, 'mode': 'markers+lines',
        } for loc in locs
    ],
    'layout': {
        'xaxis': {'title': 'Date'},
        'yaxis': {'title': (chosen_det + " (" + str(units) + ")")}
    }
}

plot(fig, filename=plot_filename)

# -----------------------------------------------------------------------------------
# DESCRIPTIVE STATISTICS:

# Descriptive statistics for chosen determinand for each sampling point:
stats = df_ordered['resultQualified'].groupby(df_ordered['sample.samplingPoint.notation']).describe()
print("Descriptive statistics table for {}:".format(chosen_det + " (" + str(units) + ")"))
display(stats)

# Write descriptive statistics to .csv file:
stats.to_csv(tbl_filename, sep=',', encoding='utf-8')

# -----------------------------------------------------------------------------------
# MAPPING:

# Get list of x-coordinates for sampling points:
eastings = df_ordered["sample.samplingPoint.easting"].unique()
# Get list of y-coordinates for sampling points:
northings = df_ordered["sample.samplingPoint.northing"].unique()
# Get list of notations for sampling points:
notations = df_ordered["sample.samplingPoint.notation"].unique()

# Convert eastings, northings to longitude, latitude:
lonlat = convert_lonlat(eastings, northings)

# Create empty list to write coordinate pairs:
coords = []
# Get each pair of coordinates from lonlat:
for i in range(0, len(lonlat[0])):
    coord = [coord[i] for coord in lonlat]
    coord.append(notations[i])
    # Print statement to manually check coordinate pair:
    # print(coord)
    # Add coordinate pair to coords list:
    coords.append(coord)
# Print statement to manually check list of coordinate pairs:
# print(coords)

# Get notation and label for each sampling point:
for coord in coords:
    df_ordered['sample.samplingPoint.easting']

# Make empty map centered on London using OpenStreetMap background:
m = folium.Map(location=[51.4772, 0], tiles="openstreetmap", zoom_start=5)

# Add markers for sampling points selected:
for coord in coords:
    folium.Marker([coord[1], coord[0]], popup=coord[2]).add_to(m)

# Save map:
m.save(map_filename)
# Display map:
display(m)

# -----------------------------------------------------------------------------------

Descriptive statistics table for Lead (ug/l):


,count,mean,std,min,25%,50%,75%,max
sample.samplingPoint.notation,,,,,,,,
NE-44100007,17.0,19.501765,10.510538,2.21,13.7000,18.400,21.9000,50.000
NE-44100013,16.0,7.121250,2.811787,4.12,4.7275,6.395,9.8575,11.800
NE-44100014,16.0,19.516250,8.926692,9.96,13.4250,16.950,22.2750,45.400
NE-44100023,36.0,14.698889,10.998278,3.47,8.8750,11.200,15.6000,62.800
NE-44100024,29.0,42.308966,110.901217,1.00,4.4200,9.350,28.6000,536.000
NE-44100027,58.0,16.347069,13.291420,3.92,8.5775,12.800,18.3750,82.800
NE-44100107,72.0,24.071111,37.541501,2.27,9.0750,17.250,28.0750,317.000
NE-44100110,20.0,10.853000,4.164796,4.38,8.2175,9.930,12.4500,19.700
NE-44100141,48.0,3.047708,2.684811,1.00,1.0000,2.365,3.5925,14.100
